Negative Sampling — это метод, который используют в задаче обучения на больших данных, особенно в нейронных сетях, для экономии вычислительных ресурсов и уменьшения сложности задачи. Наиболее популярное его применение — при обучении моделей для обработки естественного языка (NLP), таких как Word2Vec.

В Word2Vec задача заключается в обучении модели, которая предсказывает вероятность совместного появления слов. Полное предсказание для всех слов (softmax по всему словарю) будет дорогостоящим, поэтому вместо вычисления вероятности для всех слов, выбираются только несколько "отрицательных" примеров.

Давайте разберем этот процесс по шагам и реализуем его на Python.

Пример с использованием Negative Sampling в Word2Vec
Рассмотрим небольшую реализацию Skip-gram модели с Negative Sampling. Мы обучим модель предсказывать вероятность того, что пара слов (слово-контекст) является настоящей или сгенерированной (отрицательной).

Шаги:
Подготовка данных — создаем данные и словарь.
Создание положительных пар — пары из текущего слова и его контекста.
Negative Sampling — для каждой пары из положительного примера выбираем несколько отрицательных примеров.
Обучение модели — минимизируем ошибку на положительных парах и максимизируем на отрицательных.

In [1]:
import numpy as np
import random
from collections import Counter

# Пример текста
text = "we are what we repeatedly do excellence then is not an act but a habit".split()

# Словарь
vocab = list(set(text))
word_to_index = {word: i for i, word in enumerate(vocab)}
index_to_word = {i: word for i, word in enumerate(vocab)}

vocab_size = len(vocab)

print("Vocabulary:", vocab)
print("Word to Index Mapping:", word_to_index)


Vocabulary: ['a', 'but', 'are', 'an', 'what', 'do', 'repeatedly', 'then', 'excellence', 'is', 'habit', 'act', 'not', 'we']
Word to Index Mapping: {'a': 0, 'but': 1, 'are': 2, 'an': 3, 'what': 4, 'do': 5, 'repeatedly': 6, 'then': 7, 'excellence': 8, 'is': 9, 'habit': 10, 'act': 11, 'not': 12, 'we': 13}


2. Создание положительных пар (пары (слово, контекст))
Создадим пары (слово, контекст) для каждого слова в предложении с окном размером window_size.

In [2]:
# Параметры
window_size = 2

# Положительные пары
def generate_positive_pairs(text, window_size):
    positive_pairs = []
    for i, word in enumerate(text):
        target_word_idx = word_to_index[word]
        context_indices = range(max(0, i - window_size), min(len(text), i + window_size + 1))
        for j in context_indices:
            if i != j:
                context_word_idx = word_to_index[text[j]]
                positive_pairs.append((target_word_idx, context_word_idx))
    return positive_pairs

positive_pairs = generate_positive_pairs(text, window_size)
print("Positive pairs (word index, context index):", positive_pairs)


Positive pairs (word index, context index): [(13, 2), (13, 4), (2, 13), (2, 4), (2, 13), (4, 13), (4, 2), (4, 13), (4, 6), (13, 2), (13, 4), (13, 6), (13, 5), (6, 4), (6, 13), (6, 5), (6, 8), (5, 13), (5, 6), (5, 8), (5, 7), (8, 6), (8, 5), (8, 7), (8, 9), (7, 5), (7, 8), (7, 9), (7, 12), (9, 8), (9, 7), (9, 12), (9, 3), (12, 7), (12, 9), (12, 3), (12, 11), (3, 9), (3, 12), (3, 11), (3, 1), (11, 12), (11, 3), (11, 1), (11, 0), (1, 3), (1, 11), (1, 0), (1, 10), (0, 11), (0, 1), (0, 10), (10, 1), (10, 0)]


3. Negative Sampling
Теперь для каждого (слово, контекст) выбираем отрицательные примеры — слова, которые не встречаются в этом контексте. Обычно их выбирают случайно, но чаще встречающиеся слова могут появляться с большей вероятностью.

In [3]:
# Выборка отрицательных примеров
def generate_negative_samples(positive_pairs, num_negative_samples, vocab_size):
    negative_samples = []
    word_counts = Counter([pair[0] for pair in positive_pairs])
    total_count = sum(word_counts.values())
    
    # Вероятность для каждого слова
    sampling_prob = {word: count/total_count for word, count in word_counts.items()}
    
    for target_word, context_word in positive_pairs:
        negative_context_words = []
        
        while len(negative_context_words) < num_negative_samples:
            negative_sample = random.randint(0, vocab_size - 1)
            if negative_sample != context_word:
                negative_context_words.append(negative_sample)
        
        negative_samples.append((target_word, negative_context_words))
    
    return negative_samples

num_negative_samples = 3
negative_samples = generate_negative_samples(positive_pairs, num_negative_samples, vocab_size)
print("Negative samples (word index, negative context indices):", negative_samples)


Negative samples (word index, negative context indices): [(13, [3, 8, 3]), (13, [11, 11, 11]), (2, [0, 12, 8]), (2, [6, 8, 3]), (2, [1, 7, 8]), (4, [0, 4, 12]), (4, [4, 9, 6]), (4, [4, 9, 0]), (4, [1, 11, 4]), (13, [12, 0, 9]), (13, [2, 5, 1]), (13, [0, 13, 4]), (13, [11, 6, 8]), (6, [1, 3, 3]), (6, [2, 11, 12]), (6, [2, 8, 6]), (6, [10, 13, 4]), (5, [5, 11, 12]), (5, [0, 0, 7]), (5, [6, 3, 1]), (5, [0, 12, 13]), (8, [12, 13, 3]), (8, [8, 3, 1]), (8, [12, 11, 1]), (8, [12, 2, 7]), (7, [2, 12, 13]), (7, [12, 13, 13]), (7, [13, 13, 8]), (7, [0, 5, 11]), (9, [5, 1, 4]), (9, [11, 0, 6]), (9, [1, 2, 0]), (9, [13, 7, 4]), (12, [9, 9, 13]), (12, [12, 10, 7]), (12, [5, 13, 13]), (12, [2, 4, 13]), (3, [8, 11, 6]), (3, [9, 5, 4]), (3, [1, 10, 13]), (3, [13, 4, 6]), (11, [6, 11, 1]), (11, [8, 13, 12]), (11, [6, 13, 9]), (11, [10, 1, 1]), (1, [10, 0, 0]), (1, [12, 0, 10]), (1, [1, 5, 13]), (1, [7, 12, 9]), (0, [1, 6, 7]), (0, [7, 8, 7]), (0, [12, 12, 4]), (10, [6, 10, 3]), (10, [7, 4, 4])]


4. Обучение модели
Теперь, когда у нас есть положительные и отрицательные пары, обучим модель на Skip-gram с Negative Sampling. Мы будем использовать простую нейронную сеть с вложениями (embedding).

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

# Skip-gram модель с отрицательной выборкой
class SkipGramNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramNegSampling, self).__init__()
        self.target_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.context_embedding = nn.Embedding(vocab_size, embedding_dim)
    
    def forward(self, target_word, context_word, negative_samples):
        # Положительная пара
        target = self.target_embedding(target_word)  # (1, embedding_dim)
        context = self.context_embedding(context_word)  # (1, embedding_dim)
        pos_score = torch.mul(target, context).sum(dim=1)  # Складываем по embedding_dim
        pos_loss = torch.log(torch.sigmoid(pos_score))  # Лосс для положительных примеров

        # Отрицательные примеры
        neg_context = self.context_embedding(negative_samples)  # (num_neg_samples, embedding_dim)
        neg_score = torch.matmul(neg_context, target.t())  # Скалярное произведение с транспонированием
        neg_loss = torch.log(torch.sigmoid(-neg_score)).sum()  # Лосс для отрицательных примеров
        
        # Общий лосс
        return -(pos_loss + neg_loss).mean()

# Параметры
vocab_size = 100  # Пример размера словаря
embedding_dim = 10
model = SkipGramNegSampling(vocab_size, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Пример данных для обучения
positive_pairs = [(1, 2), (2, 3), (3, 4)]  # Пример пар "слово-контекст"
negative_samples = {1: [3, 4], 2: [1, 4], 3: [1, 2]}  # Пример отрицательных образцов

# Обучение
for epoch in range(10):
    total_loss = 0
    for target_word, context_word in positive_pairs:
        # Преобразование в тензоры
        target_word = torch.tensor([target_word], dtype=torch.long)
        context_word = torch.tensor([context_word], dtype=torch.long)
        neg_samples = torch.tensor(negative_samples[target_word.item()], dtype=torch.long)
        
        optimizer.zero_grad()
        loss = model(target_word, context_word, neg_samples)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch}, Loss: {total_loss:.4f}")



Epoch 0, Loss: 9.6720
Epoch 1, Loss: 8.8416
Epoch 2, Loss: 8.1147
Epoch 3, Loss: 7.4441
Epoch 4, Loss: 6.8280
Epoch 5, Loss: 6.2648
Epoch 6, Loss: 5.7515
Epoch 7, Loss: 5.2842
Epoch 8, Loss: 4.8580
Epoch 9, Loss: 4.4680


Пояснение к коду
Положительные пары: Создаем положительные пары (слово, контекст) для каждой пары слов в предложении в пределах window_size.
Отрицательные примеры: Для каждого (слово, контекст) создаем несколько отрицательных примеров, которые не являются частью контекста.
Skip-gram модель с Negative Sampling:
Для каждой положительной пары (target, context) мы вычисляем вероятность их совместного появления.
Для каждой пары (target, negative_context) мы максимизируем вероятность их раздельного появления.
Оптимизация: Обучаем модель, минимизируя ошибку, для того чтобы вложения (embedding) лучше отображали смысловые связи слов.
Результаты
Модель обучается представлять слова так, чтобы схожие слова имели близкие векторы. Negative Sampling ускоряет обучение, создавая только небольшое количество отрицательных примеров, что делает метод эффективным для задач NLP.

Пояснения
Положительные пары:

Лосс для положительных пар рассчитывается через скалярное произведение target и context, а затем проходит через sigmoid и логарифм, чтобы получить pos_loss.
Отрицательные примеры:

Используем отрицательные примеры для данного целевого слова, умножая их эмбеддинги на транспонированный вектор target. Это приводит к правильному скалярному произведению по нужной размерности, и на выходе мы получаем оценку neg_score.
После sigmoid и логарифма суммируем neg_loss для всех отрицательных примеров.
Общий лосс:

Общий лосс рассчитывается как отрицательная сумма положительных и отрицательных лоссов.